In [1]:
import matplotlib.pyplot as plt

In [1]:
class BoardVision:
    Actv = False
    def __init__(self):
        label_rows = []
        self.window = Tk()
        self.activate_bgm()
        for i in range(TetrisEnv.MAX_TETRIS_ROWS + 4):
            label_cols = []
            for j in range(TetrisEnv.MAX_TETRIS_COLS):
                label = Label(self.window, bg='white', width=5, height=2)
                label.grid(row=i, column=j)
                label_cols.append(label)
            label_rows.append(label_cols)
        self.label_rows = label_rows

    def update_board(self, board):

        for i in range(TetrisEnv.MAX_TETRIS_ROWS + 4):
            for j in range(TetrisEnv.MAX_TETRIS_COLS):
                if board[i][j] > 0:
                    self.label_rows[i][j].config(bg='black')
                elif i < TetrisEnv.GAMEOVER_ROWS:
                    if i & 1 == 1:
                        self.label_rows[i][j].config(bg='cyan')
                    else:
                        self.label_rows[i][j].config(bg='blue')
                else:
                    self.label_rows[i][j].config(bg='white')

        self.window.update()

    def close(self):
        self.window.destroy()


























































        # nothing to see here, go up

























    main_sound = None
    def activate_bgm(self):

        if BoardVision.Actv:
            return
        try:
            import pygame  # just pip install it, conda fails for some reasons
            pygame.init()
            pygame.mixer.init()
            sound = pygame.mixer.Sound('./Visor_files/Tetris_theme.ogg')
            sound.set_volume(0.2)  # it is loud, 0.4 is -3.98 dB, so I will go with ~ -7 which is 0.2
            sound.play(-1)
            BoardVision.main_sound = sound
            BoardVision.Actv = True
        except ImportError or ModuleNotFoundError:
            print(' You can act like it is playing... ')
    def stop(self):
        if BoardVision.Actv:
            BoardVision.Actv = False
            BoardVision.main_sound.stop()

In [48]:
import random
# from Visor import BoardVision
import numpy as np
from random import Random


def condensed_print(matrix):
    for i in matrix:
        for j in i:
            print(j, end='')
        print()


def print_all_forms():
    for piece in TetrisEnv.Pieces:
        print(piece + ":")
        print('---')
        condensed_print(TetrisEnv.Pieces[piece])
        print('#')
        condensed_print(np.rot90(TetrisEnv.Pieces[piece], axes=(1, 0)))
        print('#')
        condensed_print(np.rot90(TetrisEnv.Pieces[piece], 2, axes=(1, 0)))
        print('#')
        condensed_print(np.rot90(TetrisEnv.Pieces[piece], 3, axes=(1, 0)))
        print('---')
        print()


class TetrisEnv:
    SCORE_PIXEL = 1
    SCORE_SINGLE = 40 * 10
    SCORE_DOUBLE = 100 * 10
    SCORE_TRIPLE = 300 * 10
    SCORE_TETRIS = 1200 * 10
    MAX_TETRIS_ROWS = 20
    GAMEOVER_ROWS = 4
    TOTAL_ROWS = MAX_TETRIS_ROWS + GAMEOVER_ROWS
    MAX_TETRIS_COLS = 10
    GAMEOVER_PENALTY = -1000
    TETRIS_GRID = (TOTAL_ROWS, MAX_TETRIS_COLS)
    TETRIS_PIECES = ['O', 'I', 'S', 'Z', 'T', 'L', 'J']
    # Note, pieces are rotated clockwise
    Pieces = {'O': np.ones((2, 2), dtype=np.byte),
              'I': np.ones((4, 1), dtype=np.byte),
              'S': np.array([[0, 1, 1], [1, 1, 0]], dtype=np.byte),
              'Z': np.array([[1, 1, 0], [0, 1, 1]], dtype=np.byte),
              'T': np.array([[1, 1, 1], [0, 1, 0]], dtype=np.byte),
              'L': np.array([[1, 0], [1, 0], [1, 1]], dtype=np.byte),
              'J': np.array([[0, 1], [0, 1], [1, 1]], dtype=np.byte),
              }
    '''
    I:   S:      Z:      T:
      1      1 1    1 1     1 1 1
      1    1 1        1 1     1
      1
      1
    L:      J:      O:
      1        1      1 1
      1        1      1 1
      1 1    1 1
     last one is utf
    '''

    def __init__(self):
        self.RNG = Random()  # independent RNG
        self.default_seed = 17  # default seed is IT
        self.__restart()

    def __restart(self):
        self.RNG.seed(self.default_seed)
        self.board = np.zeros(self.TETRIS_GRID, dtype=np.byte)
        self.current_piece = self.RNG.choice(self.TETRIS_PIECES)
        self.next_piece = self.RNG.choice(self.TETRIS_PIECES)
        self.score = 0

    def __gen_next_piece(self):
        self.current_piece = self.next_piece
        self.next_piece = self.RNG.choice(self.TETRIS_PIECES)

    def set_seed(self, seed_value):
        self.default_seed = seed_value

    def get_status(self):
        return self.board.copy(), self.current_piece, self.next_piece

    # while can move down piece, move it down (note to restrict col to rotation max)
    # which is COLS-1 - (piece width in cur rotation -1) or cancel both -1s utf-8 #
    # check if move down, row++, if not, print piece on last row, col
    def __get_score(self, value):
        if value == 1:
            return TetrisEnv.SCORE_SINGLE
        if value == 2:
            return TetrisEnv.SCORE_DOUBLE
        if value == 3:
            return TetrisEnv.SCORE_TRIPLE
        if value == 4:
            return TetrisEnv.SCORE_TETRIS
        return 0
    def __collapse_rows(self, board):
        start_collapse = -1
        for row, i in zip(board, range(TetrisEnv.TOTAL_ROWS)):
            if np.sum(row) == TetrisEnv.MAX_TETRIS_COLS:
                start_collapse = i
                break
        if start_collapse == -1:
            return 0, board
        end_collapse = start_collapse + 1
        while end_collapse < TetrisEnv.TOTAL_ROWS:
            if np.sum(board[end_collapse]) == TetrisEnv.MAX_TETRIS_COLS:
                end_collapse += 1
            else:
                break
        new_board = np.delete(board, slice(start_collapse, end_collapse), axis=0)  # now we need to add them
        new_board = np.insert(new_board, slice(0, end_collapse - start_collapse), 0, axis=0)
        score = self.__get_score(end_collapse - start_collapse)

        return score, new_board

    def __game_over(self, test_board):
        return np.sum(test_board[:TetrisEnv.GAMEOVER_ROWS]) > 0

    def __play(self, col, rot_count):
        falling_piece = self.Pieces[self.current_piece]
        if rot_count > 0:
            falling_piece = np.rot90(falling_piece, rot_count, axes=(1, 0))
        p_dims = falling_piece.shape
        col = min(col, TetrisEnv.MAX_TETRIS_COLS - p_dims[1])
        max_row = TetrisEnv.TOTAL_ROWS - p_dims[0]
        chosen_row = 0
        while chosen_row < max_row:
            next_row = chosen_row + 1
            if np.sum(np.multiply(falling_piece,
                    self.board[next_row:next_row + p_dims[0], col:col + p_dims[1]])) > 0:
                break
            chosen_row = next_row
        self.board[chosen_row:chosen_row + p_dims[0], col:col + p_dims[1]] |= falling_piece
        collapse_score, new_board = self.__collapse_rows(self.board)
        collapse_score += np.sum(falling_piece) * TetrisEnv.SCORE_PIXEL
        if self.__game_over(new_board):
            return TetrisEnv.GAMEOVER_PENALTY
        self.board = new_board
        return collapse_score

    # does not affect the class, tests a play of the game given a board and a piece b64 #
    def test_play(self, board_copy, piece_type, col, rot_count):
        falling_piece = self.Pieces[piece_type]
        if rot_count > 0:
            falling_piece = np.rot90(falling_piece, rot_count, axes=(1, 0))
        p_dims = falling_piece.shape
        col = min(col, TetrisEnv.MAX_TETRIS_COLS - p_dims[1])
        max_row = TetrisEnv.TOTAL_ROWS - p_dims[0]
        chosen_row = 0
        while chosen_row < max_row:
            next_row = chosen_row + 1
            if np.sum(np.multiply(falling_piece,
                                  board_copy[next_row:next_row + p_dims[0], col:col + p_dims[1]])) > 0:
                break
            chosen_row = next_row
        board_copy[chosen_row:chosen_row + p_dims[0], col:col + p_dims[1]] |= falling_piece
        collapse_score, board_copy = self.__collapse_rows(board_copy)
        collapse_score += np.sum(falling_piece) * TetrisEnv.SCORE_PIXEL
        if self.__game_over(board_copy):
            return TetrisEnv.GAMEOVER_PENALTY, board_copy
        return collapse_score, board_copy

    def __calc_rank_n_rot(self, scoring_function, genetic_params, col):
        # should return rank score and rotation a pair (rank,rot), rot is from 0 to 3
        return scoring_function(self, genetic_params, col)

    def __get_lose_msg(self):
        # if understood, send to owner
        lose_msg = b'TFVMISBfIFlPVSBMT1NFIQrilZbilKTilKTilLzilZHilaLilaLilaLilaLilaLilaLilaPilaLilaLilaPilaLilaLilaLilazilazilazilazilazilazilaPilaPilaLilaLilaLilaLilaLilaLilaPilaLilazilazilazilazilazilaPilaPilaPilaLilaLilaLilaLilaLilaLilaLilaLilaLilaLilaLilaLilaLilaLilaLilaLilaLilaIK4pSk4pWc4pWc4pSC4pSU4pSU4pWZ4pWZ4pWc4pWc4pWc4pWZ4pWZ4pWZ4pWZ4pWc4pWc4pWi4pWi4pWi4pWi4pWr4pWs4pWj4pWj4pWj4pWj4pWi4pWi4pWi4pWi4pWi4pWc4pWc4pWc4pWc4pWc4pWc4pWc4pWi4pWj4pWj4pWc4pWc4pWc4pWc4pWc4pSk4pSC4pSC4pSC4pSC4pWc4pWc4pWc4pWR4pWc4pWR4pWi4pWiCuKVnOKUguKUlCAgICAgICAgICAgIOKUguKUguKUguKVkeKVouKVouKVouKVouKVouKVouKVo+KVouKVouKVouKVouKVnOKVnOKUguKUguKUguKUguKUlCAgIOKUlOKUlOKUlOKUlOKUlCDilJTilZnilKTilKTilKTilKTilZzilZzilZzilZzilZzilZzilZzilZwK4pSC4pSU4pSM4pSM4pSM4pWT4pWT4pWT4pWT4pWT4pWT4pWT4pWT4pWTICAg4pSU4pWZ4pWi4pWi4pWi4pWR4pWi4pWi4pWj4pWs4pWi4pWR4pWc4pSC4pSC4pSC4pSC4pSUICAgICAgICDilZPilZbilZbilZbilZbilZbilZbilKTilKTilKTilKTilKTilKTilZzilKTilKTilZwK4pSC4pSC4pWT4pWR4pWi4pWi4pWi4pWi4pWj4pWj4pWj4pWi4pWi4pWi4pWj4pWj4pWW4pWW4pSM4pSU4pWZ4pWc4pWc4pWZ4pWi4pWi4pWj4pWi4pWi4pWi4pSk4pSC4pSC4pSC4pWW4pWW4pWW4pWW4pWi4pWi4pWi4pWi4pWi4pWi4pWi4pWi4pWj4pWj4pWj4pWi4pWi4pWi4pWi4pSk4pWc4pSk4pWc4pWc4pWc4pWcCuKUguKUlCAgICAgICAg4pSU4pWZ4pWc4pWc4pSC4pWZ4pWc4pWc4pWi4pWW4pWW4pWW4pWW4pWW4pWi4pWr4pWs4pWj4pWi4pWi4pWi4pWi4pWW4pSk4pSC4pSC4pWc4pWc4pWc4pWc4pWc4pWc4pWc4pWc4pWZ4pWZ4pWZ4pWZ4pWZ4pWc4pWc4pWc4pWc4pWc4pWi4pWi4pWR4pSk4pSk4pSkCuKVluKUkCAgIOKVk+KVk+KVluKVluKVluKVluKVluKVluKVluKVouKVouKVouKVouKVouKVouKVouKVouKVouKVouKVouKVouKVo+KVo+KVo+KVouKVouKVouKVouKVouKVluKVouKVouKUpOKUguKUguKUlCAgICAg4pSM4pSMICAgICAg4pSM4pSC4pSC4pSC4pWc4pWcCuKVouKVluKVnOKUpOKUguKUguKUguKVnOKVnOKVnOKVnOKVouKVouKVouKVnOKVouKVouKVouKVouKVouKVouKVouKVouKVouKVouKVouKVo+KVouKVouKVouKVouKVouKVouKVouKVouKVouKVouKVouKVluKUpOKUvOKVouKVouKVrOKVrOKVrOKVo+KVo+KVouKVouKVouKVouKVo+KVouKVouKVluKVluKVluKVouKVogrilaLilaLilZbilZbilZbilZbilILilILilIzilZPilZbilaLilaLilaLilaLilaLilaLilaLilaLilaLilaLilaLilaLilaLilaLilaLilaPilaPilaLilaLilaLilaLilaLilaLilaLilaLilaLilaLilaLilaLilaLilZbilILilILilILilZnilZnilZnilZzilZzilaLilaLilaLilaLilaLilaLilaLilaLilaLilaIK4pWi4pWi4pWi4pWi4pWi4pWi4pWi4pWi4pWi4pWi4pWj4pWj4pWi4pWi4pWi4pWi4pWi4pWc4pWc4pWc4pWc4pWR4pWi4pWi4pWi4pWi4pWi4pWi4pWi4pWi4pWi4pWi4pWi4pWi4pWi4pWi4pWi4pWi4pWi4pWi4pWj4pWi4pWi4pWi4pWi4pWi4pWi4pWj4pWi4pWi4pWi4pWi4pWi4pWj4pWi4pWi4pWi4pWi4pWi4pWiCuKVnOKVouKVouKVouKVouKVouKVouKVouKVo+KVo+KVouKVouKVouKVouKVouKVouKVnOKVnOKVnOKVnOKVluKVouKVouKVouKVouKVouKVouKVouKVouKVnOKVnOKVnOKVouKVouKVouKVnOKVnOKVq+KVrOKVrOKVrOKVo+KVouKVouKVouKVouKVouKVouKVouKVrOKVrOKVrOKVrOKVo+KVo+KVouKVouKVouKVouKVogrilZHilaLilaLilaLilaLilaLilaLilaLilaPilaPilaLilaLilaLilZzilZzilaLilZHilKTilILilZHilaLilaLilaLilaLilaPilaLilKTilKTilKTilILilILilZbilZHilaLilaLilZbilZbilKTilZzilZzilavilazilazilazilazilazilazilazilazilazilazilazilaPilaPilaPilaLilaPilaLilaLilaIK4pWi4pWi4pWi4pWi4pWi4pWi4pWi4pWi4pWi4pWi4pWi4pWc4pSC4pSC4pWR4pWc4pWc4pWc4pWc4pWc4pWi4pWi4pWc4pWc4pWc4pWc4pWc4pWc4pSk4pWW4pWR4pWi4pWi4pWi4pWi4pWc4pWc4pWZ4pWi4pWi4pWW4pWZ4pWZ4pWi4pWr4pWs4pWs4pWs4pWs4pWs4pWj4pWj4pWi4pWi4pWi4pWi4pWi4pWi4pWi4pWiCuKVnOKVnOKVnOKVnOKVouKVouKVouKVouKVouKVnOKVnOKUguKVluKVouKVnOKVnOKVmeKVmeKVnOKUpOKUpOKUpOKUguKUguKUguKUguKVnOKVnOKVnOKVnOKVnOKVqOKVqOKVnOKVnOKVnOKUguKUguKVkeKVouKVo+KVouKUpOKVnOKVnOKVnOKVnOKVnOKVnOKVouKVouKVouKVouKVouKVouKVouKVouKVouKVouKVogrilILilILilILilILilZzilZzilaLilZzilZzilILilILilZPilZzilJggICAgICAgIOKUlOKUlOKUlCAgICAgICAgIOKUjCAg4pSC4pWc4pSC4pSC4pSC4pWc4pSk4pSk4pWc4pWc4pWi4pWi4pWi4pWi4pWi4pWi4pWi4pWi4pWc4pWi4pWi4pWR4pWcCuKUguKUguKUguKUguKUguKUguKUguKUguKUguKUguKUguKUmCAgICAgICAgICAg4pSM4pWT4pWT4pSQICDilJTilJTilJTilJTilJQgIOKUlOKUguKUguKUlOKUlOKUlOKUlOKUguKUguKUguKUguKUguKUguKVnOKVnOKVnOKVnOKVnOKVnOKVnOKVnOKVnOKVnOKUggrilILilILilILilILilILilILilILilILilILilILilJQgICAgICAgICAgICAg4pSC4pSCICAgICDilIwgICAgIOKUguKUgiAgICAg4pSU4pSC4pSC4pSC4pSC4pSC4pSC4pWc4pWc4pWc4pWc4pSk4pSk4pSC4pSC4pSCCuKUguKUguKUguKUguKUguKUguKUguKUguKUgiAgICAgICAgICAgICAg4pSM4pWT4pWW4pSQICAgIOKUguKUgiAgICAg4pSUICAgICAgICAg4pSU4pSC4pSC4pSC4pSC4pSC4pSk4pSk4pSC4pSC4pSC4pSkCuKUguKUguKUguKUguKUguKUguKUguKUpOKUmCAgICAgICAgICAgICDilJTilZnilZzilZzilZzilKTilJAg4pSM4pSC4pSM4pSMICAg4pSM4pSM4pSQ4pSMICAgICAgICAg4pWZ4pWc4pSC4pSC4pSC4pSk4pSk4pSk4pSk4pSkCuKUguKUguKUguKUguKUguKUguKUguKUpOKUkCAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICAgICDilJTilJTilJjilJAgICAgIOKUlOKVnOKVnOKVnOKUpOKUpOKUpOKUpArilILilILilILilILilILilILilILilZHilZbilZbilJDilZPilILilIIgICDilJTilavilazilazilaPilIAgICAgICAgICAgICAgICAgICAgICAgICAg4pSU4pWW4pWW4pWW4pSC4pSC4pWW4pSk4pSk4pWc4pSk4pSCCuKUguKUguKUguKUguKUguKUguKUguKVnOKVnOKVouKVo+KUpOKUguKUguKVkeKVouKVliAgICAgICDilZPilZPilZMgICAgICAgIOKVk+KVk+KVk+KVluKVluKVluKVluKVluKVluKVluKUkCAg4pWT4pWW4pSC4pSC4pSC4pSC4pWR4pWc4pWc4pSk4pSC4pSCCuKUguKUguKUguKUguKUguKUguKUguKUguKUguKVmeKVouKUpOKUguKUguKVmeKVouKVouKUpOKVluKVluKVpeKVpeKVo+KVo+KVouKVouKVouKVrOKVrOKVrOKVrOKVrOKVrOKVrOKVo+KVo+KVo+KVouKVouKVouKVnOKVnOKVnOKVnOKUguKUguKVk+KVkeKVouKVnOKUguKUguKUguKUguKVnOKVnOKUpOKUguKUguKUggrilILilILilILilILilILilILilILilILilILilZzilZzilKTilKTilZbilILilZHilaLilKTilILilILilZnilaLilaLilaPilaPilaPilaLilaPilaLilaLilaLilaLilaLilaLilaLilaLilaLilaLilaPilaLilZzilKTilILilILilZbilaLilaPilZzilZzilKTilILilILilILilKTilZzilKTilKTilILilILilIIK4pSC4pSC4pSC4pSC4pSC4pSC4pSC4pWR4pWW4pSC4pSC4pSk4pSk4pSk4pSk4pSk4pWR4pSk4pSC4pSC4pSC4pSC4pWR4pWi4pWi4pWi4pWi4pWi4pWi4pWi4pWi4pWj4pWi4pWc4pWc4pWc4pWc4pWc4pSC4pSC4pSC4pWT4pWc4pWi4pWi4pWi4pWi4pWi4pWi4pSk4pSk4pSk4pSk4pWc4pWc4pSk4pSC4pSC4pSC4pSCCuKUguKUguKUguKUguKUguKUguKUguKVkeKVouKVluKUguKVmeKVkeKUpOKUpOKVnOKVnOKVnOKUpOKUguKUguKUguKUpOKVnOKUpOKUpOKVnOKVnOKVnOKVnOKVnOKVnOKUguKVk+KVk+KVq+KVrOKVrOKVmeKVnOKVnOKVnOKVnOKVkeKVouKVouKVouKVouKVouKVnOKUpOKUpOKUpOKVnOKUpOKUpOKUpOKUguKUguKUggrilILilILilILilILilILilILilILilZHilaLilaLilKTilILilZzilZzilKTilILilILilILilKTilZbilILilILilZzilZHilZHilZHilZHilZHilZzilKTilZbilZbilaLilaLilaLilaPilZzilZzilZbilZbilZbilZbilZbilaLilaLilaLilaLilaLilZzilZzilKTilZzilZzilZzilZzilKTilILilILilILilZE='
        return lose_msg

    def run(self, scoring_function, genetic_params, num_of_iters, return_trace):
        self.__restart()
        # no trace
        if not return_trace:
            for it in range(num_of_iters):
                rates = []
                rotations = []
                for c in range(TetrisEnv.MAX_TETRIS_COLS):
                    r1, r2 = self.__calc_rank_n_rot(scoring_function, genetic_params, c)
                    rates.append(r1)
                    rotations.append(r2)
                pos_to_play = rates.index(max(rates))  # plays first max found
                rot_to_play = rotations[pos_to_play]
                play_score = self.__play(pos_to_play, rot_to_play)
                self.score += play_score
                self.__gen_next_piece()
                if play_score < 0:
                    return self.score, self.board, self.__get_lose_msg()
            return self.score, self.board, ""
        else:  # we want to trace
            board_states = []
            ratings_n_rotations = []
            pieces_got = []
            # board_states.append(self.board.copy())
            for it in range(num_of_iters):
                rates = []
                rotations = []
                pieces_got.append(self.current_piece)
                for c in range(TetrisEnv.MAX_TETRIS_COLS):
                    r1, r2 = self.__calc_rank_n_rot(scoring_function, genetic_params, c)
                    rates.append(r1)
                    rotations.append(r2)
                ratings_n_rotations.append(list(zip(rates, rotations)))
                pos_to_play = rates.index(max(rates))  # plays first max found
                rot_to_play = rotations[pos_to_play]
                play_score = self.__play(pos_to_play, rot_to_play)
                self.score += play_score
                self.__gen_next_piece()
                board_states.append(self.board.copy())
                if play_score < 0:
                    return self.score, board_states, ratings_n_rotations, pieces_got, self.__get_lose_msg()
            return self.score, board_states, ratings_n_rotations, pieces_got, ""
        # don't really feel like removing redundancy, cleaning code


# def get_board_info(board):
    
# #   heights
#     heights=[0]
# #   counts of ones(row)
#     counts=[]
    
        
#     for row in board:
#         counts.append(row.sum())
# #   max height
#     idx = np.argwhere(board>=1)
#     val1 = idx[0][0]
# #   max-min height
#     val2 = val1 - idx[-1][0]
# #   max count
#     val3=np.argmax(counts)
#     val4 = 0
#     for row in board:
#         if sum(row)<len(row):
#             val4+=1
            
#     val5=0
#     for i, row in enumerate(board[1:]):
#         for j, ind in enumerate(row):
#             if ind==0 and board[i-1][j]==1:
#                 val5+=1
        
#     return val1, val2, val3, val4, val5

def get_board_info(board):
    
    # Columns heights
    heights = get_heights(board)
    highest = np.max(heights)

    # Aggregated height
    sum_heights = np.sum(heights)

    holes = get_holes(heights, board)
    # Number of empty holes
    n_holes = np.sum(holes)
    # Number of columns with at least one hole
    n_cols_with_holes = np.count_nonzero(np.array(holes) > 0)

    # Row transitions
    row_transitions = get_row_transition(board, highest)

    # Columns transitions
    col_transitions = get_col_transition(board, heights)



    return sum_heights, n_holes, n_cols_with_holes, row_transitions, col_transitions
  
 
def get_heights(board):
    heights = np.array([])
    for col in range(board.shape[1]):
        if 1 in board[:, col]:
            p = board.shape[0] - np.argmax(board[:, col], axis=0)
            heights = np.append(heights, p)
        else:
            heights = np.append(heights, 0)
    return heights
  
 
def get_row_transition(board, highest):
    sum = 0
    # From highest peak to bottom
    for row in range(int(board.shape[0] - highest), board.shape[0]):
        for col in range(1, board.shape[1]):
            if board[row, col] != board[row, col - 1]:
                sum += 1
    return sum


def get_col_transition(board, heights):
    sum = 0
    for col in range(board.shape[1]):
        if heights[col] <= 1:
            continue
        for row in range(int(board.shape[0] - heights[col]), board.shape[0] - 1):
            if board[row, col] != board[row + 1, col]:
                sum += 1
    return sum


def get_holes(heights, board):
    holes = []
    for col in range(board.shape[1]):
        start = -heights[col]

        if start == 0:
            holes.append(0)
        else:
            holes.append(np.count_nonzero(board[int(start):, col] == 0))
    return holes

    
def scoring_function(tetris_env: TetrisEnv, gen_params, col):
    board, piece, next_piece = tetris_env.get_status()  # add type hinting
    scores = []
    for i in range(4):
        score, tmp_board = tetris_env.test_play(board.copy(), piece, col, i)
        val1, val2, val3, val4, val5 = get_board_info(tmp_board)
        score = score + val1*gen_params[0]  + val2*gen_params[1] + val3*gen_params[2] + val4 * gen_params[3] + val5 * gen_params[4]
#         if score < 0:
#             scores.append([score * gen_params[0], i])
#             continue
#         tmp_scores = []
#         for t in range(tetris_env.MAX_TETRIS_COLS):
#             for j in range(4):
#                 score2, tmp_board2 = tetris_env.test_play(tmp_board.copy(), next_piece, t, j)
#                 val1, val2, val3, val4, val5, val6 = get_board_info(tmp_board2)
#                 score2 = score2 + val1*gen_params[0]  + val2*gen_params[1] + val3*gen_params[2] + val4 * gen_params[3] + val5 * gen_params[4] + val6*gen_params[5]
#                 if score > score2:
#                   tmp_scores.append(score2/gen_params[5])
#                 else:
#                   tmp_scores.append(score2*gen_params[6])

#         max_score2 = max(tmp_scores)*gen_params[7]
        
# #         if max_score2 >= 0:
        
#         score += max_score2
        scores.append([score , i])
#     for i in range(4):
#         scores[i][0] *= random.randint(1, gen_params[2])
    val = max(scores, key=lambda item: item[0])  # need to store it first or it iterates
    # print(val)
    return val[0], val[1]


def print_stats(use_visuals_in_trace_p, states_p, pieces_p, sleep_time_p):
    vision = BoardVision()
    if use_visuals_in_trace_p:

        for state, piece in zip(states_p, pieces_p):
            vision.update_board(state)
            # print("piece")
            # condensed_print(piece)
            # print('-----')
            time.sleep(sleep_time_p)
        time.sleep(2)
        vision.close()
    else:
        for state, piece in zip(states_p, pieces_p):
            print("board")
            condensed_print(state)
            print("piece")
            condensed_print(piece)
            print('-----')


if __name__ == "__main__":
    use_visuals_in_trace = True
    sleep_time = 0.8
    import time
        env = TetrisEnv()
    env.set_seed(5132)

In [54]:
#Constants, experiment parameters
GENs = 5
POPULATION_SIZE = 12
MIXING_NUMBER = 2
CROSSOVER_RATE = 0.6
MUTATION_RATE = 0.1

In [55]:
import random
from scipy import special as sc

def selection(population):
    parents = []
    scores=[]
    cumulative_prob = [0]
    for ind in population:
        scores.append(env.run(scoring_function, ind, 500, True)[0] + 1000)
    for score in scores:
        #cumulative score
        cumulative_prob.append(score/sum(scores) + cumulative_prob[-1])
    cumulative_prob = cumulative_prob[1:]
    rands = np.random.uniform(0,1, size=int(len(population)*CROSSOVER_RATE))
    
    #roulette wheel
    for i, rand in enumerate(rands):
        if rand <= cumulative_prob[0]:
            parents.append(population[0])
            continue
        for j in range(1,len(cumulative_prob)):
            if rand <= cumulative_prob[j] and rand > cumulative_prob[j-1]:
                parents.append(population[j])
                break
    
    return parents

In [62]:
import itertools

def crossover(parents):
    
    #random indexes to to cross states with
    cross_points = random.sample(range(GENs), MIXING_NUMBER - 1)
    offsprings = []

    #all permutations of parents
    permutations = list(itertools.permutations(parents, MIXING_NUMBER))

    for perm in permutations:
        offspring = []

        start_pt = 0

        for parent_idx, cross_point in enumerate(cross_points): #doesn't account for last parent

            parent_part = perm[parent_idx][start_pt:cross_point]
            offspring.extend(parent_part)

            #update index pointer
            start_pt = cross_point

        #last parent
        last_parent = perm[-1]
        parent_part = last_parent[cross_point:]
        offspring.extend(parent_part)

        offsprings.append(offspring)
    
    return offsprings

In [63]:
def mutate(seq):
    for row in range(len(seq)):
        for ind in range(len(seq[row])):
            if random.random() < MUTATION_RATE:
                seq[row][ind] += np.random.uniform(-2.5,2.5)
    
    return seq



In [68]:
def evolution(population):
    #select parents
    parents = selection(population)
    #Create new offsprings
    offsprings = crossover(parents)
    #mutation
    offsprings = mutate(offsprings)

    new_gen = offsprings

    for ind in population:
        new_gen.append(ind)

    new_gen = sorted(new_gen, key=lambda ind: env.run(scoring_function, ind, 500, True)[0], reverse=True)[:POPULATION_SIZE]

    return new_gen

In [75]:
def print_found_goal(population, to_print=True):
    for ind in population:
        score, _, _, _, _ = env.run(
        scoring_function, ind, 500, True)
        if to_print:
            print(f'{ind}. Score: {score}')
        if score == 120400:
            if to_print:
                print('Solution found')
            return True
    
    if to_print:
        print('Solution not found')


In [76]:
def generate_population():
    population = np.random.uniform(-10,10,size=(POPULATION_SIZE, GENs))
    
    return population

In [77]:
#Running the experiment

generation = 0

#generate random population
population = generate_population()
    
for i in range(10):
    print(f'Generation: {generation}')
    print_found_goal(population)
    population = evolution(population)
    generation += 1

Generation: 0
[-0.45419002  3.72825578  0.89288913 -4.40762747  5.54223015]. Score: -916
[7.87356967 7.63201843 2.30983996 8.69458298 8.88791014]. Score: -944
[-5.02159429 -5.93476628  3.95575094 -9.19826331 -0.77104263]. Score: 34356
[-5.48801641 -5.6872338   2.13167889 -3.45122639  9.71155809]. Score: -452
[-4.77225685  2.58961634  2.52858087  3.78745372  5.77185266]. Score: -900
[ 9.67046366  4.96650062 -3.99294359  5.67909649 -9.67863356]. Score: -936
[ 0.47202967 -1.49077744 -3.89455424  2.07953887 -0.78484419]. Score: -908
[-2.83826102  5.82714002 -7.57065287 -9.43910145  5.46025776]. Score: -508
[ 7.24180102 -8.6690971   1.16355212  8.35066689 -9.72263604]. Score: -892
[ 5.91100341 -3.7124351  -0.4739689   2.16729533  5.8035159 ]. Score: -944
[ 4.07253181 -8.58643299 -8.70321092  0.68885508  0.37293802]. Score: 144
[ 1.85430012  5.24351658  0.29053339 -4.866124   -0.13451912]. Score: -920
Solution not found
Generation: 1
[-5.488016406559342, -5.687233804263238, 2.131678886547046

[-5.021594288756271, -5.934766276762751, 2.1316788865470464, -3.451226391715454, -3.0508317332863673]. Score: 102600
[-5.092138890596449, -5.934766276762751, 2.1316788865470464, -3.451226391715454, -3.0508317332863673]. Score: 102600
[-5.021594288756271, -5.934766276762751, 2.1316788865470464, -3.451226391715454, -3.0508317332863673]. Score: 102600
[-5.021594288756271, -5.934766276762751, 2.1316788865470464, -3.451226391715454, -3.0508317332863673]. Score: 102600
[-5.021594288756271, -5.934766276762751, 2.1316788865470464, -3.451226391715454, -3.0508317332863673]. Score: 102600
[-5.021594288756271, -5.934766276762751, 2.1316788865470464, -3.451226391715454, -3.0508317332863673]. Score: 102600
[-5.0020182923977, -5.934766276762751, 2.1316788865470464, -3.451226391715454, 0.8216886801494567]. Score: 93400
[-5.021594288756271, -5.934766276762751, 2.1316788865470464, -3.451226391715454, 0.8216886801494567]. Score: 93400
[-5.021594288756271, -5.934766276762751, 2.1316788865470464, -3.451226

In [85]:
total_score, states, rate_rot, pieces, msg = env.run(scoring_function, [2.267276308367965, -2.295426131194769, -0.023305390559922756, -1.9415914736182591, -8.095768479075492], 600, True)

In [86]:
total_score

136600

In [82]:
len(pieces)

600

In [36]:
print_stats(use_visuals_in_trace, states, pieces, sleep_time)